In [ ]:
# 单数据 node_load1

import os
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.mixture import BayesianGaussianMixture
from scipy.spatial.distance import mahalanobis
from sklearn.metrics import silhouette_score

# 设定数据路径
train_file_path = r'load1-1.csv'  # 训练文件路径
test_file_path = r'load1-2.csv'  # 测试文件路径
output_result_path = r'load1_result0.csv'  # 输出结果文件路径

columns_to_use = [
    "Time",
    "node_load1"
    
]
columns_to_use = [col for col in columns_to_use if col != "Time"]
try:
    # 1. 训练阶段 - 使用data1.csv
    print("正在训练模型...")
    # 预读取并清洗列名
    raw_df = pd.read_csv(train_file_path,usecols=columns_to_use)
    raw_df.columns = raw_df.columns.str.strip()  # 去掉空格
    columns_available = raw_df.columns

    # 找出缺失的列
    missing_columns = set(columns_to_use) - set(columns_available)
    if missing_columns:
        print("以下列在 CSV 文件中未找到：", missing_columns)
        raise ValueError("列名不匹配，检查是否拼写错误或存在空格/编码问题")

    # 只选用需要的列
    train_df = raw_df[columns_to_use]

    # train_df = pd.read_csv(train_file_path, usecols=columns_to_use)
    
    # 使用SimpleImputer填充缺失值
    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    X_train = imputer.fit_transform(train_df[columns_to_use])
    
    # 通过轮廓系数确定最佳聚类数
    print("正在通过轮廓系数确定最佳聚类数...")
    best_n = 3  # 至少2个聚类
    best_score = -1
    max_n = 10  # 尝试的最大聚类数
    
    for n in range(3, max_n + 1):
        bgmm = BayesianGaussianMixture(n_components=n, random_state=0)
        labels = bgmm.fit_predict(X_train)
        score = silhouette_score(X_train, labels)
        print(f"聚类数 n={n} 的轮廓系数: {score:.4f}")
        if score > best_score:
            best_score = score
            best_n = n
    
    n_components = best_n
    print(f"最优聚类数确定为: {n_components} (轮廓系数={best_score:.4f})")
    
    # 使用最佳聚类数训练最终模型
    bgmm = BayesianGaussianMixture(n_components=n_components, random_state=0)
    bgmm.fit(X_train)
    
    # 计算每个簇的逆协方差矩阵
    inv_covariances = [np.linalg.inv(cov) for cov in bgmm.covariances_]
    
    # 对每个簇计算马氏距离的最大值作为阈值
    mahalanobis_thresholds = []
    for i in range(n_components):
        component_data = X_train[bgmm.predict(X_train) == i]
        max_distance = np.max([
            mahalanobis(vector, bgmm.means_[i], inv_covariances[i])
            for vector in component_data
        ])
        mahalanobis_thresholds.append(max_distance)
    
    print(f"训练完成. 各簇的马氏距离阈值: {mahalanobis_thresholds}")
    
    # 2. 检测阶段 - 使用data2.csv
    print("正在检测异常点...")
    test_df = pd.read_csv(test_file_path)
    
    # 保存原始列（包括Time等所有列）
    result_df = test_df.copy()
    
    # 准备用于检测的数据（仅使用建模特征）
    X_test = imputer.transform(test_df[columns_to_use])
    
    # 为每行数据添加异常标记
    anomaly_flags = []
    for index, row in test_df.iterrows():
        vector = row[columns_to_use].values
        distances = []
        for i in range(n_components):
            mahalanobis_distance = mahalanobis(vector, bgmm.means_[i], inv_covariances[i])
            distances.append(mahalanobis_distance)
        
        min_distance = np.min(distances)
        cluster_index = np.argmin(distances)
        
        if min_distance > mahalanobis_thresholds[cluster_index] / 2:
            anomaly_flags.append(1)  # 异常
        else:
            anomaly_flags.append(0)  # 正常
    
    # 添加异常标记列
    result_df['is_anomaly'] = anomaly_flags

    # 保存结果到CSV
    result_df.to_csv(output_result_path, index=False)
    print(f"检测完成. 结果已保存到 {output_result_path}")
    print(f"异常点总数: {sum(anomaly_flags)}")

    
except Exception as e:
    print(f"程序出错: {e}")

正在训练模型...
正在通过轮廓系数确定最佳聚类数...


d:\appdownload\Anaconda\Anaconda3-2022.10\envs\new_env\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=12.
  warnings.warn(
d:\appdownload\Anaconda\Anaconda3-2022.10\envs\new_env\lib\site-packages\sklearn\mixture\_base.py:268: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


聚类数 n=3 的轮廓系数: 0.5799


d:\appdownload\Anaconda\Anaconda3-2022.10\envs\new_env\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=12.
  warnings.warn(
d:\appdownload\Anaconda\Anaconda3-2022.10\envs\new_env\lib\site-packages\sklearn\mixture\_base.py:268: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


聚类数 n=4 的轮廓系数: 0.5707


...

聚类数 n=9 的轮廓系数: 0.4396


d:\appdownload\Anaconda\Anaconda3-2022.10\envs\new_env\lib\site-packages\sklearn\mixture\_base.py:268: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(
d:\appdownload\Anaconda\Anaconda3-2022.10\envs\new_env\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=12.
  warnings.warn(


聚类数 n=10 的轮廓系数: 0.4388
最优聚类数确定为: 3 (轮廓系数=0.5799)
训练完成. 各簇的马氏距离阈值: [2.0704853041901834, 4.682089797771946, 1.9291833553922173]
正在检测异常点...


d:\appdownload\Anaconda\Anaconda3-2022.10\envs\new_env\lib\site-packages\sklearn\mixture\_base.py:268: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


检测完成. 结果已保存到 load1_result0.csv
异常点总数: 355


In [26]:
import csv

def calculate_overlap(text1, text2):
    # 把字符串按空格拆分成单词集合（可根据需要改为按字符或其它分隔）
    set1 = set(text1.strip().split())
    set2 = set(text2.strip().split())
    
    if not set1 and not set2:
        return 1.0  # 都是空的
    if not set1 or not set2:
        return 0.0  # 一个为空
    return len(set1 & set2) / len(set1 | set2)

def compare_csv_third_column(file1, file2):
    with open(file1, 'r', encoding='utf-8') as f1, open(file2, 'r', encoding='utf-8') as f2:
        reader1 = csv.reader(f1)
        reader2 = csv.reader(f2)

        total_overlap = 0
        row_count = 0

        for row1, row2 in zip(reader1, reader2):
            if len(row1) < 3 or len(row2) < 3:
                continue  # 如果某一行缺第三列就跳过
            col1 = row1[2]
            col2 = row2[2]

            overlap = calculate_overlap(col1, col2)
            total_overlap += overlap
            row_count += 1
            print(f"第{row_count}行第三列重合度: {overlap:.2%}")

        average_overlap = total_overlap / row_count if row_count else 0
        print(f"\n第三列总体平均重合度: {average_overlap:.2%}")

# 调用函数
compare_csv_third_column('load1_result0.csv', 'load1-result.csv')


第1行第三列重合度: 0.00%
第2行第三列重合度: 100.00%
第3行第三列重合度: 100.00%
第4行第三列重合度: 100.00%
第5行第三列重合度: 100.00%
第6行第三列重合度: 100.00%
第7行第三列重合度: 100.00%
第8行第三列重合度: 100.00%
第9行第三列重合度: 100.00%
第10行第三列重合度: 100.00%
第11行第三列重合度: 0.00%
第12行第三列重合度: 100.00%
第13行第三列重合度: 100.00%
第14行第三列重合度: 100.00%
第15行第三列重合度: 100.00%
第16行第三列重合度: 100.00%
第17行第三列重合度: 100.00%
第18行第三列重合度: 100.00%
第19行第三列重合度: 100.00%
第20行第三列重合度: 100.00%
第21行第三列重合度: 100.00%
第22行第三列重合度: 100.00%
第23行第三列重合度: 100.00%
第24行第三列重合度: 100.00%
第25行第三列重合度: 100.00%
第26行第三列重合度: 100.00%
第27行第三列重合度: 100.00%
第28行第三列重合度: 100.00%
第29行第三列重合度: 100.00%
第30行第三列重合度: 100.00%
第31行第三列重合度: 100.00%
第32行第三列重合度: 100.00%
第33行第三列重合度: 100.00%
第34行第三列重合度: 100.00%
第35行第三列重合度: 100.00%
第36行第三列重合度: 100.00%
第37行第三列重合度: 100.00%
第38行第三列重合度: 100.00%
第39行第三列重合度: 100.00%
第40行第三列重合度: 100.00%
第41行第三列重合度: 100.00%
第42行第三列重合度: 100.00%
第43行第三列重合度: 100.00%
第44行第三列重合度: 100.00%
第45行第三列重合度: 100.00%
第46行第三列重合度: 100.00%
第47行第三列重合度: 100.00%
第48行第三列重合度: 100.00%
第49行第三列重合度: 100.00%
第50行第三列重合度: 100.00%
第51行第三列重合度: 1

In [ ]:
# 单数据 node_load1

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.mixture import BayesianGaussianMixture
from scipy.spatial.distance import mahalanobis
from sklearn.metrics import silhouette_score

# 设定数据路径
train_file_path = r'load1-1.csv'  # 训练文件路径
test_file_path = r'load1-2.csv'  # 测试文件路径
output_result_path = r'load1_result4.csv'  # 输出结果文件路径

columns_to_use = [
    "Time",
    "node_load1"
]
columns_to_use = [col for col in columns_to_use if col != "Time"]

try:
    # 1. 训练阶段 - 使用 load1-1.csv
    print("正在训练模型...")
    raw_df = pd.read_csv(train_file_path, usecols=columns_to_use)
    raw_df.columns = raw_df.columns.str.strip()
    columns_available = raw_df.columns

    missing_columns = set(columns_to_use) - set(columns_available)
    if missing_columns:
        print("以下列在 CSV 文件中未找到：", missing_columns)
        raise ValueError("列名不匹配，检查是否拼写错误或存在空格/编码问题")

    train_df = raw_df[columns_to_use]

    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    X_train = imputer.fit_transform(train_df[columns_to_use])

    # 确定最佳聚类数
    print("正在通过轮廓系数确定最佳聚类数...")
    best_n = 3
    best_score = -1
    max_n = 10

    for n in range(3, max_n + 1):
        bgmm = BayesianGaussianMixture(n_components=n, random_state=0)
        labels = bgmm.fit_predict(X_train)
        score = silhouette_score(X_train, labels)
        print(f"聚类数 n={n} 的轮廓系数: {score:.4f}")
        if score > best_score:
            best_score = score
            best_n = n

    n_components = best_n
    print(f"最优聚类数确定为: {n_components} (轮廓系数={best_score:.4f})")

    # 使用最佳聚类数训练模型
    bgmm = BayesianGaussianMixture(n_components=n_components, random_state=0)
    bgmm.fit(X_train)

    # 计算马氏距离阈值
    inv_covariances = [np.linalg.inv(cov) for cov in bgmm.covariances_]
    mahalanobis_thresholds = []
    for i in range(n_components):
        component_data = X_train[bgmm.predict(X_train) == i]
        max_distance = np.max([
            mahalanobis(vector, bgmm.means_[i], inv_covariances[i])
            for vector in component_data
        ])
        mahalanobis_thresholds.append(max_distance)

    print(f"训练完成. 各簇的马氏距离阈值: {mahalanobis_thresholds}")

    # 2. 检测阶段 - 使用 load1-2.csv
    print("正在检测异常点...")
    test_df = pd.read_csv(test_file_path)
    result_df = test_df.copy()
    X_test = imputer.transform(test_df[columns_to_use])

    anomaly_flags = []
    for index, row in test_df.iterrows():
        vector = row[columns_to_use].values
        distances = []
        for i in range(n_components):
            d = mahalanobis(vector, bgmm.means_[i], inv_covariances[i])
            distances.append(d)

        min_distance = np.min(distances)
        cluster_index = np.argmin(distances)

        if min_distance * 3 > mahalanobis_thresholds[cluster_index]:
            anomaly_flags.append(1)
        else:
            anomaly_flags.append(0)

    # 添加异常标记列
    result_df['is_anomaly'] = anomaly_flags
    result_df['cluster'] = bgmm.predict(X_test)

    # 3. 可视化聚类结果
    print("正在绘制聚类结果图...")

    plt.figure(figsize=(12, 6))
    anomalies = result_df[result_df['is_anomaly'] == 1]
    plt.scatter(
    x=result_df.index,
    y=result_df["node_load1"],
    c=result_df["cluster"],
    cmap='tab10',
    label='Normal Points',
    s=10
)

# Anomalies (red circles)
    anomalies = result_df[result_df['is_anomaly'] == 1]
    plt.scatter(
        anomalies.index,
        anomalies["node_load1"],
        facecolors='none',
        edgecolors='red',
        s=10,
        linewidths=1.2,
        label='Anomalies'
    )

    plt.title("Clustering Results with Anomaly Markers")
    plt.xlabel("Sample Index")
    plt.ylabel("node_load1")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

except Exception as e:
    print(f"程序出错: {e}")


正在训练模型...
正在通过轮廓系数确定最佳聚类数...


d:\appdownload\Anaconda\Anaconda3-2022.10\envs\new_env\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=12.
  warnings.warn(
d:\appdownload\Anaconda\Anaconda3-2022.10\envs\new_env\lib\site-packages\sklearn\mixture\_base.py:268: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


聚类数 n=3 的轮廓系数: 0.5799


d:\appdownload\Anaconda\Anaconda3-2022.10\envs\new_env\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=12.
  warnings.warn(
d:\appdownload\Anaconda\Anaconda3-2022.10\envs\new_env\lib\site-packages\sklearn\mixture\_base.py:268: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


聚类数 n=4 的轮廓系数: 0.5707


...

d:\appdownload\Anaconda\Anaconda3-2022.10\envs\new_env\lib\site-packages\sklearn\mixture\_base.py:268: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(
d:\appdownload\Anaconda\Anaconda3-2022.10\envs\new_env\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=12.
  warnings.warn(


聚类数 n=9 的轮廓系数: 0.4396


d:\appdownload\Anaconda\Anaconda3-2022.10\envs\new_env\lib\site-packages\sklearn\mixture\_base.py:268: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(
d:\appdownload\Anaconda\Anaconda3-2022.10\envs\new_env\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=12.
  warnings.warn(


聚类数 n=10 的轮廓系数: 0.4388
最优聚类数确定为: 3 (轮廓系数=0.5799)
训练完成. 各簇的马氏距离阈值: [2.0704853041901834, 4.682089797771946, 1.9291833553922173]
正在检测异常点...


d:\appdownload\Anaconda\Anaconda3-2022.10\envs\new_env\lib\site-packages\sklearn\mixture\_base.py:268: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(
